## What is geopandas?

The [geopandas library](https://geopandas.org/) is an easy-to-use, powerful data analysis toolkit for working with geospatial data. It extends the capabilities of pandas to allow spatial operations on geometric types. The development of geopandas started in 2013 and it is now maintained by an active developer community.

geopandas is a "high-level" package, which means that it makes use of several other packages in the background. It combines the performance of powerful Python libraries such as [pandas](https://pandas.pydata.org/), [shapely](https://shapely.readthedocs.io/), [fiona](https://fiona.readthedocs.io/), and [pyproj](https://pyproj4.github.io/).

One of the most useful features of geopandas is its ability to interact with numerous geospatial data formats. It supports reading and writing data from/to formats including

- ESRI Shapefile
- GeoJSON
- KML
- GPKG
- PostGIS
- Spatialite

For a full list of supported file formats and other features, please refer to the official [geopandas documentation and reference guide](https://geopandas.org/).

## Whay is pyproj

[pyproj](https://pyproj4.github.io/pyproj/stable/) is a Python interface to the PROJ library, which is used for cartographic projections and coordinate transformations. It allows users to perform transformations between different coordinate reference systems (CRS) and provides tools for handling geospatial data. pyproj is widely used in geospatial analysis and geographic information systems (GIS) for tasks such as converting latitude and longitude coordinates to UTM coordinates, transforming data between different map projections, and more.

Key features of pyproj include:
- Support for a wide range of coordinate reference systems and transformations.
- Integration with other geospatial libraries such as geopandas and shapely.
- High performance and accuracy for geospatial computations.

## What is shapely

[shapely](https://shapely.readthedocs.io/en/stable/) is a Python library for the manipulation and analysis of planar geometric objects. It is built on top of the GEOS (Geometry Engine - Open Source) library and provides a simple and intuitive API for creating and working with geometric shapes such as points, lines, and polygons. shapely is commonly used in GIS, spatial analysis, and computational geometry applications.

Key features of shapely include:
- Creation and manipulation of geometric objects (points, lines, polygons, etc.).
- Support for geometric operations such as union, intersection, difference, and buffering.
- Integration with other geospatial libraries such as geopandas and pyproj.
- High performance and accuracy for geometric computations.

In [ ]:
# Import the necessary libraries
import numpy as np
import pandas as pd
import geopandas as gpd
from pyproj import Proj, Transformer, transform
from shapely import geometry
import matplotlib.pyplot as plt

## Defining Column Names for the XYZ File

When working with geospatial data, it is crucial to correctly define the column names based on the metadata accompanying the file. This ensures that the data is accurately interpreted and processed. For the XYZ file we downloaded, the metadata provides information about the various attributes recorded during the magnetic survey. Here are the column names defined for the magnetic survey data:

- **line**: The survey line number.
- **fid**: The fiducial number, which is a unique identifier for each record.
- **time**: The time of the survey measurement.
- **day**: The day of the year when the survey was conducted.
- **year**: The year when the survey was conducted.
- **latitude**: The latitude coordinate of the survey point.
- **longitude**: The longitude coordinate of the survey point.
- **radalt**: The radar altitude, indicating the height above the ground.
- **totmag**: The total magnetic field measurement.
- **resmag**: The residual magnetic field measurement.
- **diurnal**: The diurnal variation of the magnetic field.
- **geology**: The geological classification of the survey area.
- **resmagCM4**: The residual magnetic field corrected using the CM4 model.

These column names are essential for understanding the structure and content of the data, allowing for accurate analysis and visualization of the magnetic survey results.

In [ ]:
# Define column names for the magnetic survey data
column_names = ['line',
                'fid',
                'time',
                'day',
                'year',
                'latitude',
                'longitude',
                'radalt',
                'totmag',
                'resmag',
                'diurnal',
                'geology',
                'resmagCM4']

To import the `iron_river_mag.xyz` file using pandas, follow these steps:

1. **Ensure the file is in the correct directory**: Make sure the `iron_river_mag.xyz` file is located in the `IRON_RIVER` directory relative to your Jupyter Notebook.

2. **Define the column names**: Based on the metadata provided, define the column names for the magnetic survey data.

3. **Use `pd.read_csv` to load the data**: Use the `read_csv` function from pandas to read the file, specifying the delimiter and column names.
```

In [ ]:
# 1. Load the magnetic survey data as a pandas dataframe
magnetic_data = pd.read_csv('IRON_RIVER/iron_river_mag.xyz',
                            delim_whitespace=True,
                            names=column_names)

# 2. Display the first 10 rows of the magnetic survey data
magnetic_data.head(10)

## Defining the Coordinate System and Converting to Other Coordinate Systems

1. **Define the Projection Objects**:
    - We start by defining the projection objects for the coordinate systems we will be working with. In this case, we define two projections:
      - `prj_nad27`: North American Datum 1927 (NAD27) in latitude and longitude.
      - `prj_utm16`: Universal Transverse Mercator (UTM) zone 16N with WGS84 datum.

2. **Create a Transformer Object**:
    - We create a transformer object to handle the conversion between coordinate systems. For example, to transform coordinates from NAD27 to UTM zone 16N, we use the `Transformer.from_proj` method.

3. **Transform the Coordinates**:
    - Using the transformer object, we transform the longitude and latitude coordinates from the NAD27 system to the UTM zone 16N system. The transformed coordinates are stored as 'easting' and 'northing' in the dataframe.

4. **Display the Transformed Data**:
    - Finally, we display the first few rows of the dataframe to verify that the coordinates have been correctly transformed.

In [ ]:
# 1. Define the projection objects
prj_nad27 = Proj(proj='latlong', datum='NAD27')
prj_utm16 = Proj(proj='utm', zone=16, datum='WGS84')

# 2. Create a transformer object
transformer_object = Transformer.from_proj(prj_nad27, prj_utm16)

# 3. Transform the coordinates
magnetic_data['easting'], magnetic_data['northing'] = transformer_object.transform(magnetic_data['longitude'].values,
                                                                                   magnetic_data['latitude'].values)

# 4. Display the first 5 rows of the magnetic survey data
magnetic_data.head()

## Creating Shapely Points from Coordinates

1. **Extract Coordinates**:
    - Extract the 'easting' and 'northing' coordinates from the `magnetic_data` dataframe and store them in a list called `coordinates`.

2. **Create Shapely Points**:
    - Initialize an empty list called `shapely_points`.
    - Loop through the `coordinates` list and create Shapely `Point` objects for each coordinate pair. Append these points to the `shapely_points` list.
    - Alternatively, create the Shapely points in a single line using a list comprehension and store them in `shapely_points_alternative`.

3. **Add Shapely Points to DataFrame**:
    - Add the `shapely_points` list as a new column called 'geometry' in the `magnetic_data` dataframe.

4. **Display the DataFrame**:
    - Display the first 5 rows of the `magnetic_data` dataframe to verify that the Shapely points have been correctly added.

In [ ]:
# 1 Extract easting and northing coordinates and put them in a list
coordinates = magnetic_data[['easting', 'northing']].values.tolist()

# 2.1. Create an empty list of shapely points
shapely_points = []

# 2.2. Loop through the coordinates and create shapely points
for i in coordinates:
    shapely_points.append(geometry.Point(i))

# 2.3. An alternative way to create shapely points in a single line
shapely_points_alternative = [geometry.Point(i) for i in coordinates]

# 3. Add shapel points to the magnetic data
magnetic_data['geometry'] = shapely_points

# 4. Display the first 5 rows of the magnetic survey data
magnetic_data.head()

## Creating a GeoDataFrame and Plotting the Magnetic Survey Data

1. **Create a GeoDataFrame**:
    - Convert the `magnetic_data` DataFrame to a GeoDataFrame using the `gpd.GeoDataFrame` function.
    - Specify the 'geometry' column and the coordinate reference system (CRS) as `prj_utm16.srs`.

2. **Plot the Magnetic Survey Data**:
    - Use the `plot` method of the GeoDataFrame to visualize the magnetic survey data.
    - Display the plot using `plt.show()`.

In [ ]:
# 1. Create a geodataframe from the magnetic data
magnetic_geodata = gpd.GeoDataFrame(magnetic_data, geometry='geometry', crs=prj_utm16.srs)

# 2. Plot the magnetic geodata
magnetic_geodata.plot()
plt.show()


## Importing and Transforming Geospatial Data

1. **Import Shapefiles**:
    - Import the states, roads, towns, and water shapefiles using the `gpd.read_file` function from the `geopandas` library.

2. **Transform Coordinate Systems**:
    - Transform the coordinate systems of the imported shapefiles from WGS84 to UTM16 using the `to_crs` method. Follow the same steps as in the "Defining the Coordinate System and Converting to Other Coordinate Systems" section to set the coordinate system from WGS84 and convert to UTM.

3. **Display the DataFrame**:
    - Display the first 5 rows of of any of the imported fules to verify they have been correctly added.

In [ ]:
# Import states, roads, towns, and water shapefiles
# Example for importing states shapefile below, complete for the other shapefiles
states = gpd.read_file('ne_10m_admin_1_states_provinces/ne_10m_admin_1_states_provinces.shp')

# To do: Complete the code to import the other shapefiles
roads =
towns =
water =

In [ ]:
# Transform the states, roads, and towns shapefiles from WGS84 to UTM16
# To do: Define the projection object for WGS84 (UTM16 is already defined)
# To do: Create a transformer object
# To do: Transform the states, roads, and towns shapefiles

In [ ]:
# To do: Display the first 5 rows of the states, roads, and towns geodataframes (Optional)

## Plotting the Imported Shapefiles

1. **Plotting the Magnetic Survey Data**:
    - Use the `plot` method of the `magnetic_geodata` GeoDataFrame to visualize the magnetic survey data.
    - Customize the plot by specifying the column to color by, adding a legend, and setting the colormap.

2. **Plotting the Roads Data**:
    - Use the `plot` method of the `roads` GeoDataFrame to visualize the roads data.
    - Customize the plot by specifying the color and linewidth.

3. **Plotting the Towns Data**:
    - Use the `plot` method of the `towns` GeoDataFrame to visualize the towns data.
    - Customize the plot by specifying the color and markersize.

4. **Plotting the States Data**:
    - Use the `plot` method of the `states` GeoDataFrame to visualize the states data.
    - Customize the plot by specifying the edgecolor and linewidth.

5. **Plotting the Water Data**:
    - Use the `plot` method of the `water` GeoDataFrame to visualize the water bodies.
    - Customize the plot by specifying the color.

In [ ]:
# To do: Plot the magnetic_geodata and change the colors based on the values of any of the columns
# To do: Add plots for the states, roads, towns, and water geodataframes

## Clipping Geospatial Data to a Specific Area

When working with large geospatial datasets, it is often useful to clip the data to a specific area of interest to reduce the size and focus the analysis. Here are the steps to clip the geospatial data files to a specific area:

1. **Define the Extent of the Area of Interest**:
    - Determine the bounding box coordinates (extent) of the area you are interested in. This can be done using the `total_bounds` attribute of a GeoDataFrame or manually specifying the coordinates.
    - Optionally, you can add a buffer around the extent to include a slightly larger area.

2. **Clip the Geospatial Data**:
    - Use the `gpd.clip` function from the `geopandas` library to clip the geospatial data to the defined extent.
    - This function takes two arguments: the GeoDataFrame to be clipped and the bounding box or polygon defining the area of interest.

3. **Verify the Clipped Data**:
    - Plot any of the clipped GeoDataFrames to verify that the data has been successfully clipped to the desired area.

### Example Code

```python
# 1. Define the extent of the area of interest
# Get the extent of the magnetic data and add a buffer
magnetic_extent = magnetic_geodata.total_bounds
magnetic_extent += np.array([-40000, -40000, 40000, 40000])
print(magnetic_extent)

# 2. Clip the geospatial data to the defined extent
roads_clipped = gpd.clip(roads, magnetic_extent)

# 3. Verify the clipped data
roads_clipped.plot()
```

By following these steps, you can efficiently clip large geospatial datasets to focus on a specific area of interest, making your analysis more manageable and relevant.

In [ ]:
# To do: Define extent of the area of interest around the magnetic survey data
# To do: Clip the states, roads, towns, and water geodataframes to the extent of the area of interest.
#        Use a variable based on the original variable name + '_clipped' for the clipped geodataframes
# To do: Plot any of the clipped geodataframes

## Detailed Instructions for Making the Figure

1. **Create a Figure and Axes**:
    - Use the `plt.subplots` function from the `matplotlib.pyplot` module to create a figure and axes. Specify the figure size using the `figsize` parameter.

    ```python
    fig, ax = plt.subplots(figsize=(7.5, 5))
    ```

2. **Plot the Magnetic Survey Data**:
    - Use the `plot` method of the `magnetic_geodata` GeoDataFrame to visualize the magnetic survey data.
    - Specify the column to color by using the `column` parameter.
    - Add a legend using the `legend` parameter.
    - Set the colormap using the `cmap` parameter.
    - Adjust the marker size using the `markersize` parameter.

    ```python
    magnetic_geodata.plot(column='totmag', ax=ax, legend=True, cmap='viridis', markersize=15)
    ```

3. **Customize the Colorbar**:
    - Access the colorbar axis using `fig.axes[1]`.
    - Set the label for the colorbar using the `set_ylabel` method. Adjust the rotation and vertical alignment using the `rotation` and `va` parameters, respectively.

    ```python
    cbar_ax = fig.axes[1]
    cbar_ax.set_ylabel('Corrected Magnetic Value (nT)', rotation=270, va="bottom")
    ```

4. **Plot the Water Data**:
    - Use the `plot` method of the `water_clipped` GeoDataFrame to visualize the water bodies.
    - Specify the color using the `color` parameter.

    ```python
    water_clipped.plot(ax=ax, color='tab:blue')
    ```

5. **Plot the Roads Data**:
    - Use the `plot` method of the `roads_clipped` GeoDataFrame to visualize the roads data.
    - Specify the color using the `color` parameter.

    ```python
    roads_clipped.plot(ax=ax, color='yellow')
    ```

6. **Plot the Towns Data**:
    - Use the `plot` method of the `towns_clipped` GeoDataFrame to visualize the towns data.
    - Specify the color using the `color` parameter and the marker size using the `markersize` parameter.

    ```python
    towns_clipped.plot(ax=ax, color='tab:red', markersize=5)
    ```

7. **Add Labels to the Towns**:
    - Loop through the `towns_clipped` GeoDataFrame to extract the x and y coordinates and the labels.
    - Use the `ax.text` method to add text labels to the plot. Specify the font size using the `fontsize` parameter.

    ```python
    for x, y, label in zip(towns_clipped.geometry.x, towns_clipped.geometry.y, towns_clipped['NAME']):
        ax.text(x, y, label, fontsize=8)
    ```

8. **Plot the States Data**:
    - Use the `plot` method of the `states_clipped` GeoDataFrame to visualize the states data.
    - Specify the edge color using the `edgecolor` parameter and the line width using the `linewidth` parameter.

    ```python
    states_clipped.plot(ax=ax, color='none', edgecolor='black', linewidth=1)
    ```

9. **Customize the Axes**:
    - Set the x-axis label using the `set_xlabel` method.
    - Set the y-axis label using the `set_ylabel` method.
    - Set the x-axis limits using the `set_xlim` method.
    - Set the y-axis limits using the `set_ylim` method.

    ```python
    plt.xlabel('Easting (m)')
    plt.ylabel('Northing (m)')
    plt.xlim(magnetic_extent[0], magnetic_extent[2])
    plt.ylim(magnetic_extent[1], magnetic_extent[3])
    ```

10. **Add a Title**:
    - Set the title of the plot using the `plt.title` method.

    ```python
    plt.title('Aeromagnetic Survey of the Iron River Polygon')
    ```

11. **Display the Plot**:
    - Use the `plt.show` method to display the plot.

    ```python
    plt.show()
    ```

In [ ]:
fig, ax = plt.subplots(figsize=(7.5, 5))

magnetic_geodata.plot(column='totmag', ax=ax, legend=True, cmap='viridis', markersize=15)
cbar_ax = fig.axes[1]
cbar_ax.set_ylabel('Corrected Magnetic Value (nT)', rotation=270, va="bottom")

water_clipped.plot(ax=ax, color='tab:blue')

roads_clipped.plot(ax=ax, color='yellow')

towns_clipped.plot(ax=ax, color='tab:red', markersize=5)
for x, y, label in zip(towns_clipped.geometry.x, towns_clipped.geometry.y, towns_clipped['NAME']):
    ax.text(x, y, label, fontsize=8)

states_clipped.plot(ax=ax, color='none', edgecolor='black', linewidth=1)

plt.xlabel('Easting (m)')
plt.ylabel('Northing (m)')
plt.xlim(magnetic_extent[0], magnetic_extent[2])
plt.ylim(magnetic_extent[1], magnetic_extent[3])
plt.title('Aeromagnetic Survey of the Iron River Polygon')
plt.show()

## Deliverables

1. Check the `pandas_review.ipynb` notebook on how to use conditional selections. Write down a code snippet that filters the magnetic survey data for 'totmag' values greater than 60000 nT. Save this selection as a new dataframe and generate the same plot as above but just using the new dataframe with your selection

2. Using the steps provided in the previous cells, generate a figure for the `iron_river_rad.xyz` data. This time, focus on the 'total_count' column in the file.